In [1]:
# Demo of 3 ways of doing integration over a bivariate normal distribution
import numpy as np
from scipy import integrate
from scipy.stats import multivariate_normal

(sigmaX, sigmaY) = (0.5, 0.8)  # Bivariate parameters. (We'll center at origin and have no correlation.)
region = 1.0  # Region of interest is a unit square centered at the mean (0,0)

# Method 1: Scipy numerical integration:
def scipy_integration():
    global sigmaX, sigmaY, region
    def pdf(x,y):
        return multivariate_normal.pdf([x,y], mean=[0,0], cov=[[sigmaX**2, 0], [0, sigmaY**2]])
    probability, err = integrate.nquad(pdf, [[-region/2.0, region/2.0], [-region/2.0, region/2.0]])
    return probability

# Method 2: Numpy numerical integration:
def numpy_integration():
    global sigmaX, sigmaY, region
    INTEGRATION_STEPS = 1_500  # Sample steps along each axis. More points = more precision but longer compute time.
    # Create a grid of points over the target.  We assume shot group is centered on target – i.e., no sighting error.
    x = np.linspace(-region/2.0, region/2.0, INTEGRATION_STEPS)
    y = np.linspace(-region/2.0, region/2.0, INTEGRATION_STEPS)
    XY = np.stack(np.meshgrid(x, y), axis=-1)
    cov = np.array([[sigmaX**2, 0], [0, sigmaY**2]])
    Z = multivariate_normal.pdf(XY, cov=cov)
    return np.trapz(np.trapz(Z, x), y)

# Method 3: Monte Carlo simulation:
def monte_carlo_integration():
    global sigmaX, sigmaY, region
    simulations = 1_000_000
    X = np.random.normal(scale=sigmaX, size=simulations)
    Y = np.random.normal(scale=sigmaY, size=simulations)
    return sum(1 for s in range(simulations) if ((abs(X[s]) < region/2.0) and (abs(Y[s]) < region/2.0)))/simulations

print(f'Scipy numerical integration gives probability {scipy_integration():.1%}\n'
      f'Numpy numerical integration gives probability {numpy_integration():.1%}\n'
      f'Monte Carlo simulation gives probability {monte_carlo_integration():.1%}')


Scipy numerical integration gives probability 32.0%
Numpy numerical integration gives probability 32.0%
Monte Carlo simulation gives probability 31.9%


In [4]:
%timeit scipy_integration()
%timeit numpy_integration()
%timeit monte_carlo_integration()

67.1 ms ± 4.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
199 ms ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
367 ms ± 8.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
